In [1]:
from loadFilteredData import *
#loading all data
origImg = loadAllTopicData('original')
gingham = loadAllTopicData('gingham')
clarendon = loadAllTopicData('clarendon')
juno = loadAllTopicData('juno')
lark = loadAllTopicData('lark')
gotham = loadAllTopicData('gotham')
reyes = loadAllTopicData('reyes')

['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
Loading Topic: abbey
Loading Topic: airport_terminal
Loading Topic: amphitheater
Loading Topic: amusement_park
Loading Topic: aquarium
Loading Topic: aqueduct
Loading Topic: art_gallery
Loading Topic: assembly_line
Loading Topic: auditorium
(9000, 128, 128, 3)
['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
Loading Topic: abbey
Loading Topic: airport_terminal
Loading Topic: amphitheater
Loading Topic: amusement_park
Loading Topic: aquarium
Loading Topic: aqueduct
Loading Topic: art_gallery
Loading Topic: assembly_line
Loading Topic: auditorium
(9000, 128, 128, 3)
['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
Loading Topic: abbey
Loading Topic: airport_terminal
Loading Topic: amphithe

In [2]:
def createDataPlaces(images, trainPercentage):
    categories = len(images)

    imgList = []
    vectors = []

    testImgList = []
    testVectors = []


    #data for original image
    for c in range(categories):
        numImages = images[c].shape[0]
        print(numImages)
        numTrain = int(numImages * trainPercentage)
        imgList.append(images[c][:numTrain])

        featureVector = np.zeros((numTrain, categories))
        featureVector[:, c] = 1
        vectors.append(featureVector)
    
        #testing data
        testImgList.append(images[c][numTrain:])

        featureVector = np.zeros((numImages - numTrain, categories))
        featureVector[:, c] = 1
        testVectors.append(featureVector)


    X = np.vstack(imgList)
    y = np.vstack(vectors)

    Xtest = np.vstack(testImgList)
    ytest = np.vstack(testVectors)

    X, y = shuffle(X, y, random_state=0)
    Xtest, ytest = shuffle(Xtest, ytest, random_state=0)
    return X, y, Xtest, ytest
X, y, Xtest, ytest = createDataPlaces([origImg, clarendon, gingham, juno, lark, gotham, reyes], .9)
# X, y, Xtest, ytest = createDataPlaces([origImg, juno], .9)

9000
9000
9000
9000
9000
9000
9000


In [8]:
Xsmall = []
ysmall = []
for i in range(X.shape[0]):
    l = 0
    for j in range(0, X.shape[1], X.shape[1] // 4):
        for k in range(0, X.shape[1], X.shape[1] // 4):
            block = X[i, j : j + 32, k : k + 32]
            Xsmall.append(block)
            ysmall.append(y[i])
            l += 1
Xsmall = np.array(Xsmall)
ysmall = np.array(ysmall)
Xsmall, ysmall = shuffle(Xsmall, ysmall, random_state=0)

In [16]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LeakyReLU
import numpy as np

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), strides=2))
model.add(LeakyReLU(alpha=0.3))
model.add(Conv2D(32, (3, 3), strides = 2))
model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=3, strides =2))
model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(256))
model.add(LeakyReLU(alpha=0.3))
# model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(Xsmall, ysmall, epochs=150, batch_size=128, validation_split=0.005)

Train on 902664 samples, validate on 4536 samples
Epoch 1/150
902664/902664 [==============================] - 90s 99us/step - loss: 0.8814 - acc: 0.6675 - val_loss: 0.7589 - val_acc: 0.7134
Epoch 2/150
902664/902664 [==============================] - 94s 104us/step - loss: 0.7225 - acc: 0.7276 - val_loss: 0.7137 - val_acc: 0.7209
Epoch 3/150
902664/902664 [==============================] - 90s 99us/step - loss: 0.6695 - acc: 0.7471 - val_loss: 0.6754 - val_acc: 0.7372
Epoch 4/150
902664/902664 [==============================] - 91s 101us/step - loss: 0.6394 - acc: 0.7584 - val_loss: 0.6958 - val_acc: 0.7447
Epoch 5/150
902664/902664 [==============================] - 90s 99us/step - loss: 0.6217 - acc: 0.7650 - val_loss: 0.8066 - val_acc: 0.6944
Epoch 6/150
902664/902664 [==============================] - 90s 100us/step - loss: 0.6066 - acc: 0.7706 - val_loss: 0.6851 - val_acc: 0.7317
Epoch 7/150
902664/902664 [==============================] - 88s 98us/step - loss: 0.5963 - acc: 0.77

KeyboardInterrupt: 